In [1]:
import numpy as np
import tensorflow as tf
import random, math
import sktensor

In [7]:
## Hyper-Parameters
n = 1000 # number of samples
d = 10  # input dimension
k = 5   # hidden layer size
kappa = 2 
T = 100 # num of epochs
lr = 0.02
batch_size = 20

In [8]:
## Activation Function
def phi(h):
    return h**2 if h > 0 else 0
phi = np.vectorize(phi)

In [9]:
## Data and Params Generation

gauss_mat = np.random.normal(0.0, 0.1 , (d,k))
U, V = np.linalg.qr(gauss_mat) 

### TODO: U,V both are orthogonal and come from different gaussian matrices

diag = []
v_gt = []
v_choice = [1,-1]
for iter in range(k):
    diag.append(1+1.*iter*(kappa-1)/(k-1))
    v_gt.append(random.choice(v_choice))
    
Sigma = np.diag(diag)
W_gt = np.dot(np.dot(U, Sigma), np.transpose(V))
v_gt = np.asarray(v_gt)
train_x = []
train_y = []
for iter in range(n):
    train_x.append(np.random.normal(0.0,1.,d))
    train_y.append(np.dot(phi(np.dot(train_x[iter], W_gt)),v_gt))
train_x = np.asarray(train_x)
train_y = np.transpose(np.asarray(train_y))

In [10]:
def outer3(a,b,c):
    A = np.outer(a,b)
    B = []
    for third in c:
        B.append(A*third)
    return np.asarray(B)

def outer4(a,b,c,d):
    A = outer3(a,b,c)
    B = []
    for fourth in d:
        B.append(A*fourth)
    return np.asarray(B)

def outer3I(x):
    return outer3(x,x,x)

def outer4I(x):
    return outer4(x,x,x,x)

def specOuterI(x):
    d = len(x)
    iden = np.identity(d)
    final = np.zeros([d,d,d])
    for i in range(d):
        final += outer3(x, iden[i], iden[i]) + outer3(iden[i], x, iden[i])+ outer3(iden[i], iden[i],x)
    return final

def specOuterMat(M):
    d = np.shape(M)[0]
    ## TODO
    return np.zeros([d,d,d,d])
    
    

In [26]:
def multLnr(M, argList):
    if len(np.shape(M)) == 3:
        a,b,c = argList

        assert(np.shape(M)[0] == np.shape(a)[0])
        assert(np.shape(M)[1] == np.shape(b)[0])
        assert(np.shape(M)[2] == np.shape(c)[0])
        ## HardCoding Here :: TODO
        res = np.zeros([np.shape(a)[-1],np.shape(b)[-1] ])
        for itera in range(np.shape(a)[0]):
            for iterb in range(np.shape(b)[0]):
                for iterc in range(np.shape(c)[0]):
                    res += M[itera, iterb, iterc]*c[iterc]*np.outer(a[itera], b[iterb])
        return res
    else:
        assert(len(np.shape(M)) == 4)
        a,b,c,d = argList
        if len(np.shape(c)) == 2:
            pass
        else:
            res = np.zeros([np.shape(a)[-1],np.shape(b)[-1] ])
            for itera in range(np.shape(a)[0]):
                for iterb in range(np.shape(b)[0]):
                    for iterc in range(np.shape(c)[0]):
                        for iterd in range(np.shape(d)[0]):
                            res += M[itera, iterb, iterc]*c[iterc]*d[iterd]*np.outer(a[itera], b[iterb])
            return res
    
        

In [12]:
def prob(x):
    return math.pow(math.e, -0.5*np.dot(np.transpose(x),x))/np.sqrt(math.pow(2*math.pi, len(x)))

def getM1(X, y):
    M1 = np.zeros(d)
    for iter in range(np.shape(X)[0]):
        M1 += y[iter]*X[iter]*prob(X[iter])
    return M1

def getM2(X, y):
    M2 = np.zeros([d,d])
    for iter in range(np.shape(X)[0]):
        M2 += y[iter]*(np.outer(X[iter], X[iter]) - np.identity(d))*prob(X[iter])
    return M2

def getM3(X, y):
    M3 = np.zeros([d,d,d])
    for iter in range(np.shape(X)[0]):
        M3 += y[iter]*(outer3I(X[iter]) - specOuterI(X[iter]) )*prob(X[iter])
    return M3

def getM4(X, y):
    M4 = np.zeros([d,d,d,d])
    for iter in range(np.shape(X)[0]):
        M4 += y[iter]*(outer4I(X[iter]) - specOuterMat(np.outer(X[iter], X[iter])) + specOuterMat(np.identity(d)) )*prob(X[iter])
    return M4

In [33]:
def powMeth(P2, k):
    pass

In [31]:
def tensorInit(X, y):
    divInd = int(len(X)/3)
    
    # Partition
    X1 = X[:divInd]
    y1 = y[:divInd]

    X2 = X[divInd:2*divInd]
    y2 = y[divInd:2*divInd]
    
    X3 = X[2*divInd:]
    y3 = y[2*divInd:]
    
    ## P2 
    ## Estimating P2 as M2 :: TODO
#     alpha = np.random.normal(0.0, 0.1 , d)
    P2 = getM2(X1,y1)

    ## Power Method Implementation
    V = powMeth(P2, k)
    ## R3

    #### KCL

    ## RecMagSign

In [32]:
tensorInit(train_x, train_y)

array([[ -9.56671526e-06,   7.99149357e-06,   2.26556524e-05,
         -4.32454908e-06,  -4.46601439e-06,  -2.22900401e-06,
          4.48230322e-06,   9.13876919e-06,   9.45548315e-06,
         -2.20032182e-05],
       [  7.99149357e-06,  -1.95642951e-06,  -1.67658720e-05,
         -1.22704955e-05,  -3.09640996e-06,   1.26832541e-05,
          4.59272506e-08,  -6.22568212e-06,  -6.55393105e-06,
          7.14470631e-06],
       [  2.26556524e-05,  -1.67658720e-05,  -1.83635585e-05,
         -2.43668630e-05,   8.63044533e-06,   2.69849648e-05,
          8.77595287e-06,  -9.65382537e-06,   2.81117337e-05,
         -1.16203810e-05],
       [ -4.32454908e-06,  -1.22704955e-05,  -2.43668630e-05,
         -1.59600604e-05,   1.19334908e-05,  -1.03808557e-05,
         -5.90755833e-06,  -8.91514081e-06,  -1.01800295e-05,
          1.23759618e-05],
       [ -4.46601439e-06,  -3.09640996e-06,   8.63044533e-06,
          1.19334908e-05,   2.06746037e-06,  -4.82089225e-06,
          2.60850868e-06

In [ ]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    """
    h    = tf.square(tf.nn.relu((tf.matmul(X, w_1))))
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

In [ ]:
X = tf.placeholder("float", shape=[None, d])
y = tf.placeholder("float", shape=[None, 1])

# Weight initializations
w_1 = init_weights((d, k))
w_2 = init_weights((k, 1))

# Forward propagation
yhat  = forwardprop(X, w_1, w_2)

# Backward propagation
cost = tf.losses.mean_squared_error(y, yhat)
updates = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# Run SGD
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
epsilon = 1e-4
for epoch in range(T):
    # Train with each example
    i = 0
    for iter in range(int(n/batch_size)):
        sess.run(updates, feed_dict={X: train_x[i: i + batch_size], y: train_y[i: i + batch_size].reshape(batch_size,1)})
        i = (i + batch_size)%n
    train_accuracy = np.mean((train_y - sess.run(yhat, feed_dict={X:train_x, y:train_y.reshape(n,1)})) <= epsilon)
    print("Epoch = ", epoch+1," train Acc: ",100.*train_accuracy )